In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from ultralytics import YOLO
import torch

yaml_path = r"D:\Courses\Csc2503\proj\data_yolo_ouhands_nb\data.yaml"  # 你原来的 data.yaml
device = 0 if torch.cuda.is_available() else "cpu"

model = YOLO("yolov8n.pt")  # 建议从 COCO 预训练开始
model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=32,
    device=device,
    workers=0,
    single_cls=True,   # ⭐ 关键：把所有类别当成 1 类
    mosaic=0,
    degrees=10,
    flipud=False
)

# 评测（仍可用原 data.yaml）
model.val(data=yaml_path, split="val", device=device, single_cls=True)


Ultralytics 8.3.227  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Courses\Csc2503\proj\data_yolo_ouhands_nb\data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=False, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000208A4F7A350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [2]:
# === 环境防冲突（VSCode/Jupyter on Windows 建议）===
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

from ultralytics import YOLO
from pathlib import Path
import torch, cv2
from tqdm import tqdm

# ---- 路径配置 ----
TEST_IMG_DIR = Path(r"D:\Courses\Csc2503\proj\archive\OUHANDS_test\test\hand_data\colour")
YOLO_WEIGHTS = r"D:\Courses\Csc2503\proj\runs\detect\train3\weights\best.pt"  # 方案A训练出的单类模型
OUT_CROP_ROOT = Path(r"D:\Courses\Csc2503\proj\crops_from_yolo_singlecls\test")  # 输出的分类数据根目录

# 类别（A..K，跳过G）由文件名首字母确定
CLASSES = ['A','B','C','D','E','F','H','I','J','K']
IMG_EXTS = {".png",".jpg",".jpeg",".bmp",".tif",".tiff"}

# ---- YOLO 推理参数 ----
device = 0 if torch.cuda.is_available() else "cpu"
conf_thres = 0.25
iou_thres  = 0.6
imgsz = 640
margin = 0.15      # 裁剪时在框周围保留 15% 背景
make_square = True # 裁剪为近似正方形，适配 ResNet/Vit
resize_to = 224    # ResNet 输入尺寸

# ---- 函数：带边距裁剪 ----
def crop_with_margin(im, box, margin=0.15, make_square=True, resize_to=224):
    H, W = im.shape[:2]
    x1, y1, x2, y2 = [int(round(v)) for v in box]
    bw, bh = x2-x1, y2-y1
    x1m = max(0, int(x1 - margin*bw))
    y1m = max(0, int(y1 - margin*bh))
    x2m = min(W-1, int(x2 + margin*bw))
    y2m = min(H-1, int(y2 + margin*bh))
    if make_square:
        ww, hh = x2m - x1m, y2m - y1m
        if ww > hh:
            pad = (ww - hh)//2
            y1m = max(0, y1m - pad); y2m = min(H-1, y2m + pad)
        else:
            pad = (hh - ww)//2
            x1m = max(0, x1m - pad); x2m = min(W-1, x2m + pad)
    crop = im[y1m:y2m+1, x1m:x2m+1]
    if crop.size == 0:
        return None
    if resize_to:
        crop = cv2.resize(crop, (resize_to, resize_to), interpolation=cv2.INTER_AREA)
    return crop

# ---- 运行检测并保存裁剪 ----
model = YOLO(YOLO_WEIGHTS)
OUT_CROP_ROOT.mkdir(parents=True, exist_ok=True)

saved, no_det = 0, 0
for p in tqdm(sorted(TEST_IMG_DIR.iterdir()), desc="Detect&Crop"):
    if p.suffix.lower() not in IMG_EXTS: 
        continue
    letter = p.name[0].upper()
    if letter not in CLASSES:
        continue

    im = cv2.imread(str(p))
    if im is None: 
        continue
    H, W = im.shape[:2]

    preds = model.predict(
        source=str(p), imgsz=imgsz, conf=conf_thres, iou=iou_thres,
        device=device, stream=False, verbose=False
    )
    det = preds[0]
    if det.boxes is None or len(det.boxes) == 0:
        no_det += 1
        continue

    # 逐框保存到 test/<letter> 目录
    out_dir = OUT_CROP_ROOT / letter
    out_dir.mkdir(parents=True, exist_ok=True)

    xyxy = det.boxes.xyxy.cpu().numpy()
    for i, box in enumerate(xyxy):
        crop = crop_with_margin(im, box, margin=margin, make_square=make_square, resize_to=resize_to)
        if crop is None: 
            continue
        out_name = f"{p.stem}_{i}.jpg"
        cv2.imwrite(str(out_dir / out_name), crop)
        saved += 1

print(f"✅ Done. saved crops = {saved}, images with no detection = {no_det}")
print("Crops root:", OUT_CROP_ROOT)


Detect&Crop: 100%|██████████| 1000/1000 [00:26<00:00, 37.28it/s]

✅ Done. saved crops = 967, images with no detection = 48
Crops root: D:\Courses\Csc2503\proj\crops_from_yolo_singlecls\test


In [3]:
# 如果没装：!pip -q install scikit-learn thop

import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, f1_score
from thop import profile
from pathlib import Path

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 裁剪后的数据（上一步输出）：test/<A|B|...|K>/*.jpg
CLS_TEST_ROOT = OUT_CROP_ROOT  # 直接引用上一步的变量

# 你的 ResNet18 分类权重（整图版或裁剪版，按你训练的那个模型来）
RESNET_WEIGHTS = r"D:\Courses\Csc2503\proj\resnet18_ouhands_best.pt"  # 或 r"resnet18_raw_best.pt"

# Data transforms
eval_tf = transforms.Compose([
    transforms.ToTensor(),  # 已经是 224x224，无需再 resize/crop
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

test_ds = datasets.ImageFolder(root=str(CLS_TEST_ROOT), transform=eval_tf)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=0, pin_memory=False)
num_classes = len(test_ds.classes)
print("Classes:", test_ds.classes)

# 定义并载入 ResNet18
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
sd = torch.load(RESNET_WEIGHTS, map_location="cpu")
state_dict = sd.get("state_dict", sd)
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.load_state_dict(state_dict, strict=False)
model = model.to(device).eval()

# 推理并计算 Top-1 / Macro-F1
all_pred, all_true = [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device, non_blocking=True)
        logits = model(x)
        pred = logits.argmax(1).cpu().tolist()
        all_pred += pred
        all_true += y.tolist()

top1 = accuracy_score(all_true, all_pred)
macro_f1 = f1_score(all_true, all_pred, average="macro")
print(f"Top-1: {top1:.4f}")
print(f"Macro-F1: {macro_f1:.4f}")

# Params / FLOPs（224x224）
params_m = sum(p.numel() for p in model.parameters()) / 1e6
dummy = torch.randn(1, 3, 224, 224).to(device)
flops, _ = profile(model, inputs=(dummy,), verbose=False)
flops_g = flops / 1e9
print(f"Params (M): {params_m:.3f}")
print(f"FLOPs  (G): {flops_g:.2f}")

print(f"\n| ResNet18 on YOLO hand crops | None | {top1:.4f} | {macro_f1:.4f} | {params_m:.2f} | {flops_g:.2f} |")


Device: cuda:0
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'K']
Top-1: 0.9142
Macro-F1: 0.9147
Params (M): 11.182
FLOPs  (G): 1.82

| ResNet18 on YOLO hand crops | None | 0.9142 | 0.9147 | 11.18 | 1.82 |
